In [1]:
key = '~/work/key'
user = 'ubuntu'

with open('ansible_hosts.txt', 'r') as f:
    hosts = [l.strip() for l in f.readlines()]
master = hosts[0]
hosts_name = 'ansible_hosts.yaml'
with open(hosts_name, 'w') as f:
    f.write(f'all:\n')
    f.write(f'  vars:\n')
    f.write(f'    dist_master_addr: {master}\n')
    f.write(f'    dist_world_size: {len(hosts)}\n')
    f.write(f'  hosts:\n')
    for i, h in enumerate(hosts):
        f.write(f'    {h}:\n')
        f.write(f'      ansible_host: {h}\n')
        f.write(f'      dist_rank: {i}\n')
hosts = hosts_name

key, hosts, master

('~/work/key', 'ansible_hosts.yaml', '176.99.135.38')

In [2]:
import os
os.environ['ANSIBLE_FORKS'] = '42'
os.environ['ANSIBLE_GATHERING'] = 'explicit'

In [5]:
!rm -vf ~/.ssh/known_hosts
!ansible --private-key {key} --ssh-common-args='-o StrictHostKeyChecking=accept-new' -i {hosts} -u {user} all -m shell -a 'uname -a; df /'

176.99.135.38 | CHANGED | rc=0 >>
Linux a100x4 5.4.0-109-generic #123-Ubuntu SMP Fri Apr 8 09:10:54 UTC 2022 x86_64 x86_64 x86_64 GNU/Linux
Filesystem     1K-blocks     Used Available Use% Mounted on
/dev/vda1      121770540 14322704 107431452  12% /


In [10]:
!ansible --private-key {key} -i {hosts} -u {user} all -m debug -a 'var=dist_rank'

176.99.135.88 | SUCCESS => {
    "dist_rank": 0
}
176.99.135.111 | SUCCESS => {
    "dist_rank": 1
}
176.99.135.61 | SUCCESS => {
    "dist_rank": 2
}
176.99.135.28 | SUCCESS => {
    "dist_rank": 3
}
176.99.135.24 | SUCCESS => {
    "dist_rank": 5
}
176.99.135.123 | SUCCESS => {
    "dist_rank": 4
}
176.99.135.117 | SUCCESS => {
    "dist_rank": 7
}
176.99.135.95 | SUCCESS => {
    "dist_rank": 6
}


In [12]:
!ansible --private-key {key} -i {hosts} -u {user} all -m shell -a uptime

176.99.135.38 | CHANGED | rc=0 >>
 10:43:20 up 16 min,  1 user,  load average: 0.80, 0.97, 0.69


---

In [46]:
!ansible --private-key {key} -i {hosts} -u {user} all -m script -a setup-ic.sh

In [28]:
!ansible --private-key {key} -i {hosts} -u {user} all -m script -a setup.sh

---

In [6]:
!ansible-playbook --private-key {key} -i {hosts} -u {user} data-srv.yaml

In [30]:
#!ansible --private-key {key} -i {hosts} -u {user} all -m shell -a 'rm -Rvf /kaggle /tmp/work'

In [7]:
os.environ['DATA_SRV'] = '176.99.135.121'
!ansible-playbook --private-key {key} -i {hosts} -u {user} data-srv-cli.yaml

In [8]:
!ansible-playbook --private-key {key} -i {hosts} -u {user} data.yaml


PLAY [all] *********************************************************************

TASK [setup-ic.sh] *************************************************************
changed: [176.99.135.38]

TASK [setup-cuda.sh] ***********************************************************
changed: [176.99.135.38]

TASK [setup-torch.sh] **********************************************************
changed: [176.99.135.38]

TASK [setup.sh] ****************************************************************
changed: [176.99.135.38]

TASK [download.sh] *************************************************************
changed: [176.99.135.38]

TASK [shell] *******************************************************************
changed: [176.99.135.38]

TASK [shell] *******************************************************************
changed: [176.99.135.38]

PLAY RECAP *********************************************************************
176.99.135.38              : ok=7    changed=7    unreachable=0    failed=0    skipped=

---

In [9]:
!ansible-playbook --private-key {key} -i {hosts} -u {user} jupyter.yaml


PLAY [all] *********************************************************************

TASK [prepare] *****************************************************************
changed: [176.99.135.38]

TASK [start] *******************************************************************
changed: [176.99.135.38]

PLAY RECAP *********************************************************************
176.99.135.38              : ok=2    changed=2    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   



In [10]:
!ansible-playbook --private-key {key} -i {hosts} -u {user} jupyter-status.yaml


PLAY [all] *********************************************************************

TASK [check screen] ************************************************************
changed: [176.99.135.38]

TASK [find url] ****************************************************************
changed: [176.99.135.38]

TASK [url] *********************************************************************
ok: [176.99.135.38] => {
    "jupyter_url.stdout": "[I 2022-10-18 10:43:11.815 ServerApp]  or http://176.99.135.38:8888/lab?token=a4c5033f7282ade3f7ee844f2de8815a9702279867883bf8"
}

PLAY RECAP *********************************************************************
176.99.135.38              : ok=3    changed=2    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   



In [ ]:
!ansible --private-key {key} -i {hosts} -u {user} all -m shell -a 'screen -S jupyter -X quit'

---

In [7]:
#!ansible --private-key {key} -i {hosts} -u {user} all -m shell -a 'cat /tmp/work/jupyter.log'

In [13]:
!ansible --private-key {key} -i {hosts} -u {user} all -m shell -a 'screen -ls jupyter; ps -Af | grep jupyter'

In [34]:
!ansible --private-key {key} -i {hosts} -u {user} all -m shell -a 'screen -ls jupyter'

176.99.135.27 | FAILED | rc=1 >>
No Sockets found in /run/screen/S-ubuntu.non-zero return code


In [14]:
!ansible --private-key {key} -i {hosts} -u {user} all -m shell -a 'screen -ls'

---

In [28]:
!ansible --private-key {key} -i {hosts} -u {user} all -m shell -a 'screen -S run -X quit; rm -f ~/work/*-log.txt ~/work/*_out.ipynb'

176.99.135.88 | CHANGED | rc=0 >>
No screen session found.
176.99.135.117 | CHANGED | rc=0 >>

176.99.135.95 | CHANGED | rc=0 >>

176.99.135.111 | CHANGED | rc=0 >>

176.99.135.61 | CHANGED | rc=0 >>

176.99.135.28 | CHANGED | rc=0 >>

176.99.135.123 | CHANGED | rc=0 >>

176.99.135.24 | CHANGED | rc=0 >>



In [27]:
#os.environ['RUN_NB'] = 'test.ipynb'
os.environ['RUN_NB'] = 'FaceId_multigpu_distributed.ipynb'
os.environ['RUN_WD'] = 'work'
!ansible-playbook --private-key {key} -i {hosts} -u {user} run.yaml


PLAY [all] *********************************************************************

TASK [copy] ********************************************************************
ok: [176.99.135.28] => (item=run.sh)
ok: [176.99.135.24] => (item=run.sh)
ok: [176.99.135.61] => (item=run.sh)
ok: [176.99.135.95] => (item=run.sh)
ok: [176.99.135.123] => (item=run.sh)
ok: [176.99.135.117] => (item=run.sh)
ok: [176.99.135.111] => (item=run.sh)
ok: [176.99.135.88] => (item=run.sh)
ok: [176.99.135.28] => (item=run.py)
ok: [176.99.135.117] => (item=run.py)
ok: [176.99.135.61] => (item=run.py)
ok: [176.99.135.123] => (item=run.py)
ok: [176.99.135.24] => (item=run.py)
ok: [176.99.135.88] => (item=run.py)
ok: [176.99.135.111] => (item=run.py)
ok: [176.99.135.95] => (item=run.py)
changed: [176.99.135.28] => (item=FaceId_multigpu_distributed.ipynb)
changed: [176.99.135.61] => (item=FaceId_multigpu_distributed.ipynb)
changed: [176.99.135.123] => (item=FaceId_multigpu_distributed.ipynb)
changed: [176.99.135.117] => (

In [16]:
!ansible --private-key {key} -i {hosts} -u {user} all -m shell -a 'screen -ls run'

176.99.135.61 | CHANGED | rc=0 >>
33mThere is a screen on:
	5564.run	(10/08/2022 12:49:56 PM)	(Detached)
1 Socket in /run/screen/S-ubuntu.
176.99.135.24 | CHANGED | rc=0 >>
33mThere is a screen on:
	5665.run	(10/08/2022 12:49:56 PM)	(Detached)
1 Socket in /run/screen/S-ubuntu.
176.99.135.88 | FAILED | rc=1 >>
No Sockets found in /run/screen/S-ubuntu.non-zero return code
176.99.135.117 | CHANGED | rc=0 >>
33mThere is a screen on:
	5490.run	(10/08/2022 12:49:55 PM)	(Detached)
1 Socket in /run/screen/S-ubuntu.
176.99.135.123 | CHANGED | rc=0 >>
33mThere is a screen on:
	5517.run	(10/08/2022 12:49:55 PM)	(Detached)
1 Socket in /run/screen/S-ubuntu.
176.99.135.28 | CHANGED | rc=0 >>
33mThere is a screen on:
	5487.run	(10/08/2022 12:49:56 PM)	(Detached)
1 Socket in /run/screen/S-ubuntu.
176.99.135.111 | CHANGED | rc=0 >>
33mThere is a screen on:
	5537.run	(10/08/2022 12:49:55 PM)	(Detached)
1 Socket in /run/screen/S-ubuntu.
176.99.135.95 | CHANGED | rc=0 >>
33mThere is a screen on:
	5402.run

In [17]:
!ansible --private-key {key} -i {hosts} -u {user} all -m shell -a 'tail ~/work/*-log.txt'

176.99.135.123 | CHANGED | rc=0 >>
+ out=
+ [ -n  ]
+ basename FaceId_multigpu_distributed.ipynb .ipynb
+ out=FaceId_multigpu_distributed_out.ipynb
+ [ -z 1 ]
+ python run.py FaceId_multigpu_distributed.ipynb FaceId_multigpu_distributed_out.ipynb .
176.99.135.88 | FAILED | rc=1 >>
tail: cannot open '/home/ubuntu/work/*-log.txt' for reading: No such file or directorynon-zero return code
176.99.135.24 | CHANGED | rc=0 >>
+ out=
+ [ -n  ]
+ basename FaceId_multigpu_distributed.ipynb .ipynb
+ out=FaceId_multigpu_distributed_out.ipynb
+ [ -z 1 ]
+ python run.py FaceId_multigpu_distributed.ipynb FaceId_multigpu_distributed_out.ipynb .
176.99.135.95 | CHANGED | rc=0 >>
+ out=
+ [ -n  ]
+ basename FaceId_multigpu_distributed.ipynb .ipynb
+ out=FaceId_multigpu_distributed_out.ipynb
+ [ -z 1 ]
+ python run.py FaceId_multigpu_distributed.ipynb FaceId_multigpu_distributed_out.ipynb .
176.99.135.117 | CHANGED | rc=0 >>
+ out=
+ [ -n  ]
+ basename FaceId_multigpu_distributed.ipynb .ipynb
+ out=FaceI

In [17]:
!ansible --private-key {key} -i {hosts} -u {user} all -m shell -a 'rm -f ~/work/*-log.txt ~/work/*_out.ipynb; ls -l ~/work/'

176.99.135.94 | CHANGED | rc=0 >>
total 112
-rwxr-xr-x 1 ubuntu ubuntu 104876 Oct  6 10:36 FaceId_multigpu_distributed.ipynb
-rwxr-xr-x 1 ubuntu ubuntu    713 Oct  6 10:36 run.py
-rwxr-xr-x 1 ubuntu ubuntu    219 Oct  6 10:36 run.sh
176.99.135.40 | CHANGED | rc=0 >>
total 112
-rwxr-xr-x 1 ubuntu ubuntu 104876 Oct  6 10:36 FaceId_multigpu_distributed.ipynb
-rwxr-xr-x 1 ubuntu ubuntu    713 Oct  6 10:36 run.py
-rwxr-xr-x 1 ubuntu ubuntu    219 Oct  6 10:36 run.sh
176.99.135.60 | CHANGED | rc=0 >>
total 112
-rwxr-xr-x 1 ubuntu ubuntu 104876 Oct  6 10:36 FaceId_multigpu_distributed.ipynb
-rwxr-xr-x 1 ubuntu ubuntu    713 Oct  6 10:36 run.py
-rwxr-xr-x 1 ubuntu ubuntu    219 Oct  6 10:36 run.sh
176.99.135.115 | CHANGED | rc=0 >>
total 112
-rwxr-xr-x 1 ubuntu ubuntu 104876 Oct  6 10:36 FaceId_multigpu_distributed.ipynb
-rwxr-xr-x 1 ubuntu ubuntu    713 Oct  6 10:36 run.py
-rwxr-xr-x 1 ubuntu ubuntu    219 Oct  6 10:36 run.sh
176.99.135.61 | CHANGED | rc=0 >>
total 100
-rwxr-xr-x 1 ubuntu ubu

In [15]:
!ansible --private-key {key} -i {hosts} -u {user} all -m shell -a 'screen -S run -X quit'

176.99.135.94 | CHANGED | rc=0 >>

176.99.135.60 | CHANGED | rc=0 >>

176.99.135.61 | FAILED | rc=1 >>
No screen session found.non-zero return code
176.99.135.121 | CHANGED | rc=0 >>

176.99.135.115 | CHANGED | rc=0 >>

176.99.135.40 | CHANGED | rc=0 >>

176.99.135.97 | CHANGED | rc=0 >>

176.99.135.38 | CHANGED | rc=0 >>



---